# Modelo definitivo Educación Continua

## Tabla de contenidos
    1. Carga librerías y base de datos
    2. Manejo de bases de datos
    3. Construcción modelo definitivo
    4. Plan piloto

## Carga librerías y base de datos

In [ ]:
%pip install pytrends

In [ ]:
%pip install unidecode

In [ ]:
%pip install pyspark

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import datetime as dt
import unidecode
from pytrends.request import TrendReq
import math
from datetime import datetime
# from flask import request
import time
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
# import statsmodels.api as sm

plt.style.use('ggplot')

# /mnt/analiticastorage_educacion-continua
# Cambiar directorio de trabajo #
# os.chdir('/Users/JUANGARCESC/Documents/OneDrive - Universidad de Los Andes/2021/008-21 Modelo EDCO/Bases')

# Cargar base de datos con direcciones #
base=pd.read_csv('/content/ConsultaGT.csv')

base=base.drop_duplicates(['Consecutivo.del.curso']).reset_index()

# Transformaciones a variables de la base
base['Valor.por.hora']=pd.to_numeric(base['Valor.por.hora'].str.replace('[^\w\s]', ''))
base['Curso.nuevo'] = base['Curso.nuevo'].replace({'Sí':1, 'No':0})

# Dimensiones de la base de datos
base.shape

<command-1713951692700465>:39: FutureWarning: The default value of regex will change from True to False in a future version.
 base['Valor.por.hora']=pd.to_numeric(base['Valor.por.hora'].str.replace('[^\w\s]', ''))
Out[11]: (1310, 82)

## Manejo de bases de datos

In [ ]:
# Variables continuas:
feature_cols = ['maDesemp', 'maIPC', 'maTasaInt', 'maTRM', 'maGT_EduCon', 'maGT_Coursera', 'maGT_Cursos', 
                'maGT_Cert', 'maGT_platzi', 'maGT_udemy', 'trafico', 'clics', 'CTR', 'CPC', 'campanias', 
                'Envios', 'Aperturas', 'Clics', 'TasaApertura', 'TasaClicEn', 'TasaClicAp', 'NumCursos',
                'NumCursosFac', 'Keywords', 'Valor.por.hora']

# Variables categóricas:
feature_cat=['Exitoso2', 'Facultad', 'Modalidad', 'Categoría', 'Curso.nuevo', 'Programa', 'Consecutivo.del.curso', 'Nombre.del.curso']

# Establecer la base con variables a incluir
X = base[feature_cols+feature_cat]

# Eliminar valores vacíos en variables de marketing 
X=X[X['trafico'].notna()]

## Imputar info a variables con NA
# Variables a reemplazar con 0
var0=['CTR','CPC','campanias', 'Envios', 'Aperturas', 'Clics', 'TasaApertura','Keywords']
X[var0] = X[var0].replace({np.nan:0})

# Variables a imputar con media
varM=['Valor.por.hora']
X[varM]=X[varM].fillna(X[varM].mean())

# Dummies de variables categoricas
Categoria_dummies = pd.get_dummies(X.Categoría, prefix='Cat')
Categoria_dummies.drop(Categoria_dummies.columns[0], axis=1, inplace=True)

Modalidad_dummies = pd.get_dummies(X.Modalidad, prefix='Mod')
Modalidad_dummies.drop(Modalidad_dummies.columns[0], axis=1, inplace=True)

Facultad_dummies = pd.get_dummies(X.Facultad, prefix='Fac')
Facultad_dummies.drop(Facultad_dummies.columns[0], axis=1, inplace=True)


#X = pd.concat([X.drop(columns=['Facultad', 'Modalidad', 'Categoría']), Categoria_dummies, Modalidad_dummies, Facultad_dummies], axis=1)

# Eliminar filas completamente vacías
X=X.dropna()
features_col = ['Curso.nuevo', 'Programa', 'NumCursos', 'NumCursosFac', 'Keywords',
       'Valor.por.hora', 'Categoría', 'Modalidad', 'Facultad']
X_2 = X[features_col].copy()

y = X.Exitoso2
X = X.drop(columns=['Exitoso2'])

In [ ]:
# Dividir la base de datos en entrenamiento y prueba
random.seed(2022)
X_train, X_test, y_train, y_test = train_test_split(X_2, y, random_state=2022)

## Construcción modelo definitivo

In [ ]:
#Transformacion OHE
ct = ColumnTransformer([('ohe', OneHotEncoder(handle_unknown = 'ignore'), ['Categoría', 'Modalidad', 'Facultad'])], remainder='passthrough')

# Clasificador
rfc = RandomForestClassifier(n_estimators=100, random_state = 2022)

# Construcción del pipeline
pipe = Pipeline([('ctransformer', ct), ('clf', rfc)])
pipe = pipe.fit(X_train,y_train)

In [ ]:
X_test

Out[21]:

,Curso.nuevo,Programa,NumCursos,NumCursosFac,Keywords,Valor.por.hora,Categoría,Modalidad,Facultad
1089,1,1,56,6,32.196429,56375.000000,"Arte, Ciencias y Humanidades",Virtual,ARTES Y HUMANIDADES
35,0,1,24,3,7.642857,62964.000000,Bienestar - Desarrollo Personal,Presencial,CIENCIAS SOCIALES
781,1,1,67,4,0.000000,65476.000000,Actualización Profesional,Virtual,ECONOMIA
684,1,1,66,15,38.690476,64951.430837,Lenguas,Presencial,CENTRO DE LENGUAS
949,1,1,41,2,57.142857,55000.000000,Actualización Profesional,Virtual,MEDICINA
...,...,...,...,...,...,...,...,...,...
399,1,1,28,11,50.523810,64951.430837,Lenguas,Presencial,CENTRO DE LENGUAS
220,1,1,37,7,18.633333,71903.000000,Actualización Profesional,Presencial,DERECHO
1220,0,1,39,7,24.547619,72188.000000,Actualización Profesional,Virtual,DERECHO
863,1,1,27,7,20.464286,60867.000000,Actualización Profesional,Virtual,CIENCIAS SOCIALES


In [ ]:
X.loc[X_test.index, X_test.columns]

Out[22]:

,Curso.nuevo,Programa,NumCursos,NumCursosFac,Keywords,Valor.por.hora,Categoría,Modalidad,Facultad
1089,1,1,56,6,32.196429,56375.000000,"Arte, Ciencias y Humanidades",Virtual,ARTES Y HUMANIDADES
35,0,1,24,3,7.642857,62964.000000,Bienestar - Desarrollo Personal,Presencial,CIENCIAS SOCIALES
781,1,1,67,4,0.000000,65476.000000,Actualización Profesional,Virtual,ECONOMIA
684,1,1,66,15,38.690476,64951.430837,Lenguas,Presencial,CENTRO DE LENGUAS
949,1,1,41,2,57.142857,55000.000000,Actualización Profesional,Virtual,MEDICINA
...,...,...,...,...,...,...,...,...,...
399,1,1,28,11,50.523810,64951.430837,Lenguas,Presencial,CENTRO DE LENGUAS
220,1,1,37,7,18.633333,71903.000000,Actualización Profesional,Presencial,DERECHO
1220,0,1,39,7,24.547619,72188.000000,Actualización Profesional,Virtual,DERECHO
863,1,1,27,7,20.464286,60867.000000,Actualización Profesional,Virtual,CIENCIAS SOCIALES


In [ ]:
pipe.score(X_test, y_test)

Out[15]: 0.7584097859327217

In [ ]:
temp_pdf = X_test.copy()
y_pred = pipe.predict_proba(temp_pdf)
temp_pdf[['Consecutivo.del.curso', 'Nombre.del.curso']] = X.iloc[X_test.index, -2:]
temp_pdf['P1'] = y_pred[:, 1]
temp_pdf['Real'] = y_test
temp_pdf.to_csv('/content/Prediccion_Validacion_sin.csv', sep=';', encoding='utf-8')

In [ ]:
X_test.columns

Out[56]: Index(['Curso.nuevo', 'Programa', 'NumCursos', 'NumCursosFac', 'Keywords',
 'Valor.por.hora', 'Categoría', 'Modalidad', 'Facultad'],
 dtype='object')

In [ ]:
print(classification_report(y_test, pipe.predict(X_test)))

precision recall f1-score support

 0 0.80 0.86 0.83 222
 1 0.64 0.55 0.59 105

 accuracy 0.76 327
 macro avg 0.72 0.70 0.71 327
weighted avg 0.75 0.76 0.75 327

In [ ]:
pipe.steps[1][1].n_features_in_

Out[58]: 33

In [ ]:
# predictions = pipe.predict_proba(base[features_col])
# base['probability_1'] = predictions[:, 1]
# df = base[['Curso.nuevo', 'Programa', 'NumCursos', 'NumCursosFac', 'Keywords',
#        'Valor.por.hora', 'Categoría', 'Modalidad', 'Facultad',
#        'Consecutivo.del.curso', 'Nombre.del.curso', 'probability_1', 'Exitoso2']]

In [ ]:
# df.to_csv('/dbfs/mnt/analiticastorage_educacion-continua/99Prueba/Predicciones_Entrenamiento.csv', index=False, sep=';')

## Plan Predicción nuevos registros

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.shell import spark

In [1]:
#Limpieza cache spark
spark.catalog.clearCache()

NameError: ignored

In [2]:
cat_df = spark.createDataFrame(X_test[['Facultad', 'Categoría', 'Modalidad']])
cat_df = cat_df.distinct()

NameError: ignored

In [ ]:
#Importar utilidades de pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

#Lectura cursos a predecir
predict_df = pd.read_csv('/content/EstructuraBaseTest.csv', delimiter=';', encoding='utf-8')
predict_df = spark.createDataFrame(predict_df)
for column in predict_df.columns:
  predict_df = predict_df.withColumnRenamed(column, column.replace('.', '_'))

In [ ]:
#Casteo de fechas
predict_df = predict_df.withColumn('Fecha_creación', F.to_date(F.col('Fecha_creación'), 'dd/MM/yyyy'))
predict_df = predict_df.withColumn('Fecha_inicio', F.to_date(F.col('Fecha_inicio'), 'dd/MM/yyyy'))

#Casteo de decimales
predict_df = predict_df.withColumn('Valor_por_hora', F.regexp_replace(F.col('Valor_por_hora'), ',', '.').cast('float'))

#Agregar ventanas de tiempo
predict_df = predict_df.withColumn('Fecha_busqueda', F.add_months(F.col('Fecha_creación'), -3))
predict_df = predict_df.withColumn('Fecha_ventana', F.add_months(F.col('Fecha_creación'), -12))

In [ ]:
#Separación de keywords
predict_df = predict_df.withColumn('Keywords', F.regexp_replace(F.col('Keywords'), r',\s', ','))
predict_df = predict_df.withColumn('Keywords', F.split(F.col('Keywords'), ','))
predict_df = predict_df.withColumn('Keywords', F.array_remove(F.col('Keywords'), ''))

In [3]:
predict_df = predict_df.cache()
predict_df.count()

NameError: ignored

In [ ]:
display(predict_df)

In [ ]:
#Completar información con pytrens - Google Trends
def get_google_trend(keywords, start_date, end_date):
  start_date_str = start_date.strftime('%Y-%m-%d')
  end_date_str = end_date.strftime('%Y-%m-%d')
  keywords_list = list(keywords)
  keywords_list = [kw.lower().strip().replace(r'[^\w\s]', '') for kw in keywords_list]
  print(keywords_list)
  df = None
  max_tol = 30
  init_tol = 1
  pytrend = TrendReq(hl='su', tz=360, geo='CO')
  try:
    pytrend.build_payload(keywords_list[:5], timeframe=f'{start_date_str} {end_date_str}', cat=958)
    df = pytrend.interest_over_time().drop(['isPartial'],axis=1)
  except:
    request_success = False
    while (not request_success) and (init_tol < max_tol):
      time.sleep(1+round(random.random(), 2))
      try:
        pytrend.build_payload(keywords_list[:5], timeframe=f'{start_date_str} {end_date_str}', cat=958)
        df = pytrend.interest_over_time().drop(['isPartial'],axis=1)
        request_success = True
      except:
        request_success = False
        init_tol = init_tol + 1
  if df is None:
    return 0
  means = df.mean()
  return round(float(means.mean()), 2)

In [ ]:
df_rows = predict_df.collect()

In [ ]:
predict_df = predict_df.withColumn('Keywords_gt', F.lit(0))
for row in df_rows:
  keywords = get_google_trend(row['Keywords'], row['Fecha_busqueda'], row['Fecha_creación'])
  predict_df = predict_df.withColumn('Keywords_gt', F.when(F.col('Consectutivo curso') == row['Consectutivo curso'], keywords).otherwise(F.col('Keywords_gt')))

In [ ]:
predict_df = (
  predict_df
  .withColumn('Keywords_list', F.array_join(F.col('Keywords'), ', '))
  .withColumn('Keywords', F.col('Keywords_gt'))
  .withColumn('Curso_nuevo', F.when(F.col('Curso_nuevo') == 'Si', 1).otherwise(0))
  .withColumn('Programa', F.when(F.col('Programa') == 'Si', 1).otherwise(0))
  .select(
    F.col('Curso_nuevo').alias('Curso.nuevo'), F.col('Programa'),
    F.col('NumCursos'), F.col('NumCursosFac'), F.col('Keywords'),
    F.col('Valor_por_hora').alias('Valor.por.hora'), F.col('Categoría'),
    F.col('Modalidad'), F.col('Facultad'), F.col('Consectutivo curso'),
    F.col('Keywords_list'), F.col('Nombre')
  )
)

In [ ]:
display(predict_df)

In [ ]:
predict_df = predict_df.toPandas()
df_to_predict = predict_df[features_col]

In [ ]:
predictions = pipe.predict_proba(df_to_predict)

In [ ]:
predict_df['probability_1'] = predictions[:, 1]

In [ ]:
#Predicho
display(predict_df)

In [ ]:
predict_df.to_csv('/content/Predicciones_sin.csv', sep=';', index=False)

In [ ]:
#Predicciones base de entrenamiento y validacion
base_df = base[list(X_2.columns) + ['Consecutivo.del.curso', 'Nombre.del.curso', 'Exitoso2']]
base_df = base_df.dropna()
predicciones = pipe.predict_proba(base_df[X_2.columns])
base_df['probability_1'] = predicciones[:, 1]

In [ ]:
base_df.to_csv('/content/Predicciones_Entrenamiento_sin.csv', sep=';', index=False)

In [ ]:
def make_keywords(x):
  x_list = list(x)
  x_list = [i for i in x_list if not isinstance(i, float)]
  return ', '.join(x_list)

In [ ]:
base['Keywords_org'] = base[[f'kw{i+1}' for i in range(5)]].apply(make_keywords, axis=1)
columns_to_take = features_col + ['Fecha.de.inicio', 'Fecha.de.creación', 'Keywords_org', 'Consecutivo.del.curso', 'Exitoso2', 'Nombre.del.curso'] + [f'kw{i+1}' for i in range(5)]
exitosos = base[(base['Exitoso2'] == 1) & (base['Valor.por.hora'].notna())][columns_to_take].head()
no_exitosos = base[(base['Exitoso2'] == 0) & (base['Valor.por.hora'].notna())][columns_to_take].head()
df_to_append = pd.concat([exitosos, no_exitosos])

In [ ]:
#Original
predictions = pipe.predict_proba(df_to_append[features_col])
df_to_append['probability_1'] = predictions[:, 1]
display(df_to_append[features_col + ['Consecutivo.del.curso', 'Nombre.del.curso', 'probability_1']])

In [ ]:
# df_to_append = spark.createDataFrame(df_to_append)

In [ ]:
# df_to_append = df_to_append.withColumn('Fecha.de.inicio', F.date_format(F.to_date('`Fecha.de.inicio`', 'yyyy-MM-dd'), 'dd/MM/yyyy'))
# df_to_append = df_to_append.withColumn('Fecha.de.creación', F.date_format(F.to_date('`Fecha.de.creación`', 'yyyy-MM-dd'), 'dd/MM/yyyy'))
# df_to_append = df_to_append.withColumn('Keywords', F.col('Keywords_org'))

In [ ]:
# df_to_append.toPandas().to_csv('/dbfs/mnt/analiticastorage_educacion-continua/99Prueba/BV_Conjunto_Entrenamiento_Original.csv', sep=';', index=False)

In [ ]:
# df_to_append.select(
#     '`Consecutivo.del.curso`','`Nombre.del.curso`','`Fecha.de.inicio`',
#     '`Fecha.de.creación`','`Curso.nuevo`','Programa',
#     'NumCursos','NumCursosFac','Keywords','`Valor.por.hora`',
#     'Categoría','Facultad','Modalidad'
#   ).toPandas().to_csv('/dbfs/mnt/analiticastorage_educacion-continua/99Prueba/BV_Conjunto_Entrenamiento_Adaptado.csv', sep=';', index=False)